In [3]:
# Unzip images to a custom data folder
!unzip -q /content/data.zip -d /content/custom_data

In [4]:
!ls -lh /content/
!file /content/data.zip


total 20M
drwxr-xr-x 4 root root 4.0K Mar 17 10:41 custom_data
-rw-r--r-- 1 root root  20M Mar 17 10:41 data.zip
drwxr-xr-x 1 root root 4.0K Mar 13 13:31 sample_data
/content/data.zip: Zip archive data, at least v2.0 to extract, compression method=store


In [5]:
!pip install ultralytics


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 949.2/949.2 kB 58.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 98.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 87.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 97.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [7]:

from pathlib import Path
import random
import os
import shutil

# Set the dataset path manually if not using argparse
data_path = "/content/custom_data"  # Change this to your actual dataset path
train_percent = 0.8  # Adjust this if needed

# Check for valid entries
if not os.path.isdir(data_path):
    print(f"Directory '{data_path}' not found. Verify the path and try again.")
    exit()

if not (0.01 <= train_percent <= 0.99):
    print("Invalid train_pct. Please enter a value between 0.01 and 0.99.")
    exit()

val_percent = 1 - train_percent

# Define paths
input_image_path = os.path.join(data_path, "images")
input_label_path = os.path.join(data_path, "labels")

# Define output paths
output_base = os.getcwd()  # Change this if needed
train_img_path = os.path.join(output_base, "data/train/images")
train_txt_path = os.path.join(output_base, "data/train/labels")
val_img_path = os.path.join(output_base, "data/validation/images")
val_txt_path = os.path.join(output_base, "data/validation/labels")

# Create folders if they don't already exist
for path in [train_img_path, train_txt_path, val_img_path, val_txt_path]:
    os.makedirs(path, exist_ok=True)

# Get list of all images
img_file_list = list(Path(input_image_path).rglob("*.[jpJP][pnPN][gG]"))  # Filters only images
random.shuffle(img_file_list)  # Shuffle once instead of picking randomly in a loop

# Split data
train_num = int(len(img_file_list) * train_percent)
train_files = img_file_list[:train_num]
val_files = img_file_list[train_num:]

print(f"Total images: {len(img_file_list)}")
print(f"Train images: {len(train_files)}")
print(f"Validation images: {len(val_files)}")

# Function to copy files
def copy_files(files, dest_img_folder, dest_label_folder):
    for img_path in files:
        img_fn = img_path.name
        txt_fn = img_path.stem + ".txt"
        txt_path = os.path.join(input_label_path, txt_fn)

        shutil.copy(img_path, os.path.join(dest_img_folder, img_fn))
        if os.path.exists(txt_path):  # Only copy label if it exists
            shutil.copy(txt_path, os.path.join(dest_label_folder, txt_fn))

# Copy train files
copy_files(train_files, train_img_path, train_txt_path)

# Copy validation files
copy_files(val_files, val_img_path, val_txt_path)

print("Data splitting complete.")


Total images: 126
Train images: 100
Validation images: 26
Data splitting complete.


In [8]:
import yaml
import os

def create_data_yaml(path_to_classes_txt, path_to_data_yaml):
    # Ensure classes.txt exists
    if not os.path.exists(path_to_classes_txt):
        print(f'Error: classes.txt file not found at {path_to_classes_txt}')
        return

    # Read class names
    try:
        with open(path_to_classes_txt, 'r', encoding='utf-8') as f:  # Ensure UTF-8 encoding
            classes = [line.strip() for line in f.readlines() if line.strip()]
    except Exception as e:
        print(f"Error reading classes.txt: {e}")
        return

    # Ensure we have at least one class
    if not classes:
        print("Error: classes.txt is empty. Add class names and try again.")
        return

    # Create data dictionary
    data = {
        'path': os.path.abspath('/content/data'),
        'train': 'train/images',
        'val': 'validation/images',
        'nc': len(classes),
        'names': classes
    }

    # Write to YAML file
    try:
        with open(path_to_data_yaml, 'w', encoding='utf-8') as f:  # Ensure UTF-8 encoding
            yaml.dump(data, f, sort_keys=False, allow_unicode=True)  # Allow Unicode characters
        print(f'Success: Created config file at {path_to_data_yaml}')
    except Exception as e:
        print(f"Error writing data.yaml: {e}")

# Define paths and run function
path_to_classes_txt = '/content/custom_data/classes.txt'
path_to_data_yaml = '/content/data.yaml'

create_data_yaml(path_to_classes_txt, path_to_data_yaml)

# Display file contents
print('\nFile contents:\n')
!cat /content/data.yaml

Success: Created config file at /content/data.yaml

File contents:

path: /content/data
train: train/images
val: validation/images
nc: 15
names:
- water_bottle
- Spoon
- Porcelain
- coffee_table
- Cup
- Plate
- mobile
- Kettle
- Knife
- Tri
- Slipper
- Pot
- cofee_pot
- Bucket
- Shoes


In [9]:
!yolo task=detect mode=train data=/content/data.yaml model=yolov8s.pt epochs=60 imgsz=640


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
100% 21.5M/21.5M [00:00<00:00, 395MB/s]
Ultralytics 8.3.91 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/content/data.yaml, epochs=60, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=Fal

In [10]:



!yolo detect predict model=runs/detect/train/weights/best.pt source=data/validation/images save=True

Ultralytics 8.3.91 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 11,131,389 parameters, 0 gradients, 28.5 GFLOPs

image 1/45 /content/data/validation/images/010af7a1-images_31.jpg: 640x640 1 Cup, 16.3ms
image 2/45 /content/data/validation/images/02cc4b11-photo_47_2025-03-16_15-13-33.jpg: 640x480 1 Porcelain, 1 coffee_table, 1 cofee_pot, 1 Bucket, 48.6ms
image 3/45 /content/data/validation/images/02dbe958-photo_44_2025-03-16_15-13-33.jpg: 640x480 1 Kettle, 1 Pot, 12.2ms
image 4/45 /content/data/validation/images/0abcc72e-photo_45_2025-03-16_15-13-33.jpg: 640x480 1 Porcelain, 1 coffee_table, 12.2ms
image 5/45 /content/data/validation/images/0c5070ee-il_570xN.6186657189_ag7g.png: 512x640 1 Kettle, 46.6ms
image 6/45 /content/data/validation/images/177207c9-photo_8_2025-03-16_15-13-33.jpg: 640x480 1 Cup, 1 cofee_pot, 1 Bucket, 12.9ms
image 7/45 /content/data/validation/images/18f6f8b9-photo_40_2025-03-16_15-13-33.jpg: 640x480 2 coffee_table

In [12]:
import glob
from IPython.display import Image, display
for image_path in glob.glob(f'/content/runs/detect/predict/*.jpg')[:25]:
  display(Image(filename=image_path, height=400))
  print('\n')

Output hidden; open in https://colab.research.google.com to view.

# Setting up github and push it on to the github

In [ ]:
# Set up Git
!git config --global user.name "user name"
!git config --global user.email "email@gmail.com"


In [17]:
!git clone https://github.com/Tsegaye16/Yolo-project.git


Cloning into 'Yolo-project'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (3/3), done.


In [22]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [24]:
!ls -l /content/drive/MyDrive/'Colab Notebooks'/


total 73
-rw------- 1 root root 71119 Mar 17 11:39 'Training_YOLO model.ipynb'
-rw------- 1 root root  1939 Jan 23  2024  Untitled0.ipynb
-rw------- 1 root root   850 Mar 16 14:51  Untitled1.ipynb


In [ ]:
!mv "/content/drive/MyDrive/Colab Notebooks/your_notebook.ipynb" /content/Yolo-project/
